In [12]:
import json
from simple_salesforce import Salesforce, SalesforceLogin
import requests
import io
import time
import base64
from datetime import datetime, timedelta
from collections import OrderedDict


In [13]:
sf = Salesforce(username="velocity-integration@mensatech.com.au",password="&[8Hn4baZ8/u]_%007", security_token="SUQ9nMAwQmmHCGAWpganRXg35")

session_id, instance = SalesforceLogin(username="velocity-integration@mensatech.com.au", password="&[8Hn4baZ8/u]_%007", security_token="SUQ9nMAwQmmHCGAWpganRXg35")


In [14]:
def processExistingOrder(inflowOrder, salesforceOrder,sfProducts):
    print(inflowOrder)
    if inflowOrder['timestamp'] != salesforceOrder['Timestamp__c']:
        #UPDATE salesOrder where external id = infowOrder["orderId"]
        orderId = updateOrder(inflowOrder)
        for lineItem in inflowOrder['lines']:
            sfOrderItem = getSFOrder(id)#Get  salesforceOrderItems.get(lineItem['id'])

            if not sfOrderItem:
                print(f'Product {lineItem["productId"]} was not found on SF')
                print(f'Creating {lineItem["productId"]} in SF')
                productInfo = getInflowProduct(lineItem["productId"])
                # INSERT into products with productInfo
                createdProduct = createProduct(productInfo)
                print(f'Created product: {createdProduct}')
                pass
            if lineItem['timestamp'] != sfOrderItem['timestamp']:
                # UPDATE saleOrderItem 
                updateLineItems(lineItem,sfProducts)
                pass
    return

In [15]:
def updateLineItems(item, sfProducts):
    print("Updating line item")

    # Extracting product ID from sfProducts
    productId = ""
    for product in sfProducts:
        if product["Product_Id__c"] == item["productId"]:
            productId = product["Id"]
            break

    # Extracting other line item details
    description = item.get("description", "")
    quantity = item.get("quantity", {}).get("standardQuantity", 0)
    orderValue = item.get("unitPrice", 0)
    lineItemID = item.get("salesOrderLineId", "")
    timestamp = item.get("timestamp", "")

    # Printing line item details
    print("Product:", productId)
    print("Description:", description)
    print("Quantity:", quantity)
    print("Order Value:", orderValue)
    print("LineItemID: ", lineItemID)
    print("Timestamp: ", timestamp)

    # Check for non-zero quantity before updating
    if quantity > 0:
        updatedOrderItem = sf.OrderItem.update({
            "Id": lineItemID,  # Assuming this is the identifier for the line item in Salesforce
            "Product2Id": productId,
            "Description": description,
            "Quantity": quantity,
            "UnitPrice": orderValue,
            "Timestamp": timestamp,
            # Include any other fields that need to be updated
        })
        print("Updated Order Item: ", updatedOrderItem)
    else:
        print("Skipping update for zero quantity line item")


In [16]:
def createProduct(productInfo,sfProducts):
    timestamp = ""
    if "timestamp" in productInfo:
        timestamp = productInfo["timestamp"]

    productId = ""
    if "productId" in productInfo:
        productId = productInfo["productId"]

    isActive = ""
    if "isActive" in productInfo:
        isActive = productInfo["isActive"]

    name = ""
    if "name" in productInfo:
        name = productInfo["name"]

    description = ""
    if "description" in productInfo:
        description = productInfo["description"]

    print("Timestamp:", timestamp)
    print("ProductId:", productId)
    print("isActive:", isActive)
    print("Name:", name)
    print("Description: ", description)

    productToCreate = {
        "timestamp__c": timestamp,
        "Product_Id__c":productId,
        "isActive":isActive,
        "Name":name,
        "Description":description,
        "isActive":True,
    }
    newProduct = sf.Product2.create(productToCreate)
    newProductId = newProduct.get("id")
    productToCreate["Id"] = newProductId

    print("Product created")
    print(newProduct)
    sfProducts.append(productToCreate)

    priceBookValues = {
        "Product2Id" :newProduct["id"], 
        "UnitPrice":productInfo["defaultPrice"]["unitPrice"],
        "Pricebook2Id":"01s5j00000Q9jTAAAZ",
        "IsActive": True
    }
    print("PriceBook Entry")
    print(priceBookValues)
    priceBook = sf.PricebookEntry.create(priceBookValues)
    return priceBook


In [17]:
def productExists(productId, sfProducts):
    print("Product we are looking for: ", productId)
    if sfProducts == None or len(sfProducts) < 1:
        return False
    for product in sfProducts:
        print("Current Product: ", product["Product_Id__c"])
        if product["Product_Id__c"] == productId:
            print("Found Product")
            return True
        else: 
            continue
    return False


In [18]:
def updateOrder(order):
    print("Updating an Order")

    # Extracting order details
    salesOrderId = order.get("salesOrderId", "")
    orderNumber = order.get("orderNumber", "")
    orderDate = order.get("orderDate", "")
    dueDate = order.get("dueDate", "")
    total = order.get("total", 0)
    amountPaid = order.get("amountPaid", 0)
    timestamp = order.get("timestamp", "")
    poNumber = order.get("poNumber", "")
    paidDate = order.get("paidDate", "")
    accountId = getSFAccount(order.get("customerId", ""))

    # Printing extracted order details
    print("Sales Order Id:", salesOrderId)
    print("Order Reference Number:", orderNumber)
    print("Effective Date:", orderDate)
    print("End Date:", dueDate)
    print("Total Amount:", total)
    print("Amount Paid:", amountPaid)
    print("Timestamp:", timestamp)
    print("PO Number:", poNumber)
    print("Activated Date:", paidDate)
    print("Account Id:", accountId)

    # Update the existing order
    updatedOrder = sf.Order.update({
        "Sales_Order_Id__c": salesOrderId,
        "OrderReferenceNumber": orderNumber,
        "EffectiveDate": orderDate,
        "EndDate": dueDate,
        "Pricebook2Id": "01s5j00000Q9jTAAAZ",  # Example Pricebook2Id, replace with actual ID if necessary
        "TotalAmount": total,
        "Paid__c": amountPaid,
        "Timestamp__c": timestamp,
        "PO__c": poNumber,
        "ActivatedDate": paidDate,
        "AccountId": accountId
    })

    print("Updated Order:", updatedOrder)

In [19]:
def getSFAccount(id):
    getAccountQuery = f"select Id,name from Account where Customer_Reference_Id__c = '{id}'"
    print("\nGet Account query: ", getAccountQuery)
    getAccountResult = sf.query_all(getAccountQuery)["records"]
    print("\nGet Account Results: ", getAccountResult[0])
    if len(getAccountResult) < 1:
        print("\nGet Account returnted nothing")
        return None
    
    return getAccountResult[0]


In [20]:
def getSFOrder(id):
    getOrdersQuery = f"select Id, Sales_Order_Id__c,Timestamp__c  from Order  where Sales_Order_Id__c = '{id}'"
    print("\nGet Orders query: ", getOrdersQuery)
    getOrdersResult = sf.query_all(getOrdersQuery)["records"]
    if len(getOrdersResult) < 1:
        print("\nGet Order returnted nothing")
        return None
    print("\nGet Orders Results: ", getOrdersResult[0])
    return getOrdersResult

In [21]:
inflowOrder = [{"amountPaid": "112.02000", "balance": "0.00000", "billingAddress": {"address1": "135A Coode St", "address2": "", "city": "South Perth", "state": "WA", "country": "Australia", "postalCode": "6151", "remarks": "", "addressType": "null"}, "contactName": "Van", "dueDate": "2013-07-09T12:11:36.943+08:00", "email": "", "inventoryStatus": "fulfilled", "InvoiceDate": "null", "isCancelled": "false", "isCompleted": "true", "isInvoiced": "true", "isPrioritized": "false", "lines": [{"salesOrderLineId": "5facd02a-0ba0-4e05-bcb3-fcf22e9bfe9c", "description": "Toilet brush", "discount": {"value": "0.00000", "isPercent": "true"}, "isDiscarded": "false", "productId": "739f2977-a017-4106-9374-c37a2d0d9d98", "quantity": {"standardQuantity": "3.0000", "uomQuantity": "3.0000", "uom": "", "serialNumbers": []}, "returnDate": "null", "serviceCompleted": "null", "subTotal": "5.67000", "tax1Rate": "10.00000", "tax2Rate": "0.00000", "taxCodeId": "584d8a57-1a5d-4184-8f9d-e1ac94cf8f2b", "timestamp": "00000000000B42CF", "unitPrice": "1.89000"}, {"salesOrderLineId": "158cdabe-2ff9-4c2e-b3c6-375fcd25ba12", "description": "Aluminium \r\nFoil 10m", "discount": {"value": "0.00000", "isPercent": "true"}, "isDiscarded": "false", "productId": "cf93cf4a-e429-4173-900c-ef85d051446d", "quantity": {"standardQuantity": "10.0000", "uomQuantity": "10.0000", "uom": "", 
"serialNumbers": []}, "returnDate": "null", "serviceCompleted": "null", "subTotal": "13.20000", "tax1Rate": "10.00000", "tax2Rate": "0.00000", "taxCodeId": "584d8a57-1a5d-4184-8f9d-e1ac94cf8f2b", "timestamp": "00000000000B42D0", "unitPrice": "1.32000"}, {"salesOrderLineId": "cdd21dc6-cff1-41e2-ae79-124ce4f6f1b3", "description": "10 in 1 Scouring Pad", "discount": {"value": "0.00000", "isPercent": "true"}, "isDiscarded": "false", "productId": "1aeb62f4-55e8-41d1-b920-dd01ded4da83", "quantity": {"standardQuantity": "5.0000", "uomQuantity": "5.0000", "uom": "", "serialNumbers": []}, "returnDate": "null", "serviceCompleted": "null", "subTotal": "6.90000", "tax1Rate": 
"10.00000", "tax2Rate": "0.00000", "taxCodeId": "584d8a57-1a5d-4184-8f9d-e1ac94cf8f2b", "timestamp": "00000000000B42D1", "unitPrice": "1.38000"}, {"salesOrderLineId": "efe96847-658c-48a5-b88d-3f29e9c189f1", "description": "5 in 1 T-Shape Sponge Scouring Pad", "discount": {"value": "0.00000", "isPercent": "true"}, "isDiscarded": "false", "productId": "7998db97-149c-4964-90b6-83f8b2c6286c", "quantity": {"standardQuantity": "5.0000", "uomQuantity": "5.0000", "uom": "", "serialNumbers": []}, "returnDate": "null", "serviceCompleted": "null", "subTotal": "7.85000", "tax1Rate": "10.00000", "tax2Rate": "0.00000", "taxCodeId": "584d8a57-1a5d-4184-8f9d-e1ac94cf8f2b", "timestamp": "00000000000B42D2", "unitPrice": "1.57000"}, {"salesOrderLineId": "f73d0c8d-d6aa-4a73-9a2d-032bf6756513", "description": "2 in 1 Cleaning Brush", "discount": {"value": "0.00000", "isPercent": "true"}, "isDiscarded": "false", "productId": "fefe0999-244f-4ca6-95bc-07db3facede8", "quantity": {"standardQuantity": "5.0000", "uomQuantity": "5.0000", "uom": "", "serialNumbers": []}, "returnDate": "null", "serviceCompleted": "null", "subTotal": "4.80000", "tax1Rate": "10.00000", "tax2Rate": "0.00000", "taxCodeId": "584d8a57-1a5d-4184-8f9d-e1ac94cf8f2b", "timestamp": "00000000000B42D3", "unitPrice": "0.96000"}, {"salesOrderLineId": "1473ff19-0543-4d7b-8aeb-c429244c10c7", "description": "Basket KW1007", "discount": {"value": "0.00000", "isPercent": "true"}, "isDiscarded": "false", "productId": "1d0622c1-cddf-42e9-9775-c4744d5f4f12", "quantity": {"standardQuantity": "2.0000", "uomQuantity": "2.0000", "uom": "", "serialNumbers": []}, "returnDate": "null", "serviceCompleted": "null", "subTotal": "2.62000", "tax1Rate": "10.00000", "tax2Rate": "0.00000", "taxCodeId": "584d8a57-1a5d-4184-8f9d-e1ac94cf8f2b", "timestamp": "00000000000B42D4", "unitPrice": "1.31000"}, {"salesOrderLineId": "6ca48886-5073-48d1-a9d2-3e8634ce0b7d", "description": "Leak Basin Set KW7710", "discount": {"value": "0.00000", "isPercent": "true"}, "isDiscarded": "false", "productId": "865e1830-f10e-4b15-9b33-d4555b04dfb1", "quantity": {"standardQuantity": "2.0000", "uomQuantity": "2.0000", "uom": "", "serialNumbers": []}, "returnDate": "null", "serviceCompleted": "null", "subTotal": "5.12000", "tax1Rate": "10.00000", "tax2Rate": "0.00000", "taxCodeId": "584d8a57-1a5d-4184-8f9d-e1ac94cf8f2b", "timestamp": "00000000000B42D5", "unitPrice": "2.56000"}, {"salesOrderLineId": "8bee2114-7d8a-4403-b434-0af63bdc88d7", "description": "Universal Adapter", "discount": {"value": "0.00000", "isPercent": "true"}, "isDiscarded": "false", "productId": "4ae5d5e1-b50a-43a8-9367-b1131c96fa9b", "quantity": {"standardQuantity": "5.0000", "uomQuantity": "5.0000", "uom": "", "serialNumbers": []}, "returnDate": "null", "serviceCompleted": "null", "subTotal": "9.40000", "tax1Rate": "10.00000", "tax2Rate": "0.00000", "taxCodeId": "584d8a57-1a5d-4184-8f9d-e1ac94cf8f2b", "timestamp": "00000000000B42D6", "unitPrice": "1.88000"}, {"salesOrderLineId": "aa74e861-8897-4a3c-b25d-a6721c3e13c1", "description": "Playing cards 2PK", "discount": {"value": "0.00000", "isPercent": "true"}, "isDiscarded": "false", "productId": "839a0224-87b4-4f28-91a2-3c88a424bcef", "quantity": {"standardQuantity": "12.0000", "uomQuantity": "12.0000", "uom": "", "serialNumbers": []}, "returnDate": "null", "serviceCompleted": 
"null", "subTotal": "21.60000", "tax1Rate": "10.00000", "tax2Rate": "0.00000", "taxCodeId": "584d8a57-1a5d-4184-8f9d-e1ac94cf8f2b", "timestamp": "00000000000B42D7", "unitPrice": "1.80000"}, {"salesOrderLineId": "311a2aa2-a2e7-412f-a216-431400c33b45", "description": "Pick-up reaching tool", "discount": {"value": "0.00000", "isPercent": "true"}, "isDiscarded": "false", "productId": "0ba36bcd-855f-482e-b8c9-efbc9e59b289", "quantity": {"standardQuantity": "2.0000", "uomQuantity": "2.0000", "uom": "", "serialNumbers": []}, "returnDate": "null", "serviceCompleted": "null", "subTotal": "3.00000", "tax1Rate": "10.00000", "tax2Rate": "0.00000", "taxCodeId": "584d8a57-1a5d-4184-8f9d-e1ac94cf8f2b", "timestamp": "00000000000B42D8", "unitPrice": "1.50000"}, {"salesOrderLineId": "4fd58fb3-bce2-4348-93e0-d95d727b1db2", "description": "Masking Tapes 24mm 2PK", "discount": {"value": "0.00000", 
"isPercent": "true"}, "isDiscarded": "false", "productId": "834600b5-5432-4c6c-adc7-600a155ca13c", "quantity": {"standardQuantity": "4.0000", "uomQuantity": "4.0000", "uom": "", "serialNumbers": []}, "returnDate": "null", "serviceCompleted": "null", "subTotal": "7.28000", "tax1Rate": "10.00000", "tax2Rate": "0.00000", "taxCodeId": "584d8a57-1a5d-4184-8f9d-e1ac94cf8f2b", "timestamp": "00000000000B42D9", "unitPrice": "1.82000"}, {"salesOrderLineId": "3416af23-6f98-405f-a793-48d390a909a3", "description": "Packing Tape 48mm x 50m", "discount": {"value": "0.00000", "isPercent": "true"}, "isDiscarded": "false", "productId": "5f8bbafd-9fce-43a8-ac6a-e8c75c3c74fe", "quantity": {"standardQuantity": "5.0000", "uomQuantity": "5.0000", "uom": "", "serialNumbers": []}, "returnDate": "null", "serviceCompleted": "null", "subTotal": "5.00000", "tax1Rate": "10.00000", "tax2Rate": "0.00000", "taxCodeId": "584d8a57-1a5d-4184-8f9d-e1ac94cf8f2b", "timestamp": "00000000000B42DA", "unitPrice": "1.00000"}, {"salesOrderLineId": "56cef379-7701-4f1d-b7ec-5a20191b9901", "description": "2PK Packing Tapes with dispenser", "discount": {"value": "0.00000", "isPercent": "true"}, "isDiscarded": "false", "productId": "8ddcb907-fe56-4917-8dc6-182ae7c80012", "quantity": {"standardQuantity": "4.0000", "uomQuantity": "4.0000", "uom": "", "serialNumbers": []}, "returnDate": "null", "serviceCompleted": "null", "subTotal": "9.40000", "tax1Rate": "10.00000", "tax2Rate": "0.00000", "taxCodeId": "584d8a57-1a5d-4184-8f9d-e1ac94cf8f2b", "timestamp": "00000000000B42DB", "unitPrice": "2.35000"}], "orderDate": "2013-07-05T00:00:00+08:00", "orderFreight": "null", "orderNumber": "S000593", "orderRemarks": "\r", "paymentStatus": "paid", "phone": "(08)9367 2298", "requestedShipDate": "null", "shippingAddress": 
{"address1": "135A Coode St", "address2": "", "city": "South Perth", "state": "WA", "country": "Australia", "postalCode": "6151", "remarks": "", "addressType": "null"}, "subTotal": "101.84000", "tax1": "10.18000", "tax1Name": "", "total": "112.02000", "timestamp": "00000000003E9E73", "salesOrderId": "119cc65f-4e62-4cdb-8bc7-94049006eaa7", "customer": {"customerId": "70437d64-cdaa-4bd6-b138-fce00978b080", "contactName": "Van", "customFields": {"custom1": "", "custom2": "", "custom3": "", "custom4": "", "custom5": "", "custom6": "", "custom7": "", "custom8": "", "custom9": "", "custom10": ""}, "defaultBillingAddressId": "d1ecde8e-fd81-41f5-bbd8-331a19a94ab3", "defaultCarrier": "", "defaultLocationId": "null", "defaultPaymentMethod": "", "defaultPaymentTermsId": "043de7dc-adc0-40ee-a84b-1cdb44ad76d9", "defaultSalesRep": "", "defaultSalesRepTeamMemberId": "null", "defaultShippingAddressId": "d1ecde8e-fd81-41f5-bbd8-331a19a94ab3", "discount": "0.00", "email": "", "fax": "", "isActive": "true", "lastModifiedById": "e41123ca-fcdd-446e-a3bd-9241b70a6788", "name": "South Perth Family Store", "phone": "(08)9367 2298", "pricingSchemeId": "806dbb1c-7c76-469c-b5de-9a309fd6d90f", "remarks": "", "taxExemptNumber": "", "taxingSchemeId": "null", "timestamp": "00000000003E5760", "website": ""}}]

sfOrder = [OrderedDict([('attributes', OrderedDict([('type', 'Order'), ('url', '/services/data/v57.0/sobjects/Order/8015j0000093ZrUAAU')])), ('Id', '8015j0000093ZrUAAU'), ('Sales_Order_Id__c', '119cc65f-4e62-4cdb-8bc7-94049006eaa7'), ('Timestamp__c', '23')])]

sfProducts = [OrderedDict([('attributes', OrderedDict([('type', 'Product2'), ('url', '/services/data/v57.0/sobjects/Product2/01t5j00000FRzZAAA1')])), ('Id', '01t5j00000FRzZAAA1'), ('Product_Id__c', '739f2977-a017-4106-9374-c37a2d0d9d98'), ('Name', 'HW0015'), ('timestamp__c', '0000000000CF6491'), ('IsActive', True), ('Description', 'Toilet brush')]), OrderedDict([('attributes', OrderedDict([('type', 'Product2'), ('url', '/services/data/v57.0/sobjects/Product2/01t5j00000FRzZFAA1')])), ('Id', '01t5j00000FRzZFAA1'), ('Product_Id__c', 'cf93cf4a-e429-4173-900c-ef85d051446d'), ('Name', 'KW0019'), ('timestamp__c', '000000000047A00D'), ('IsActive', True), ('Description', 'Aluminium Foil 30cm x10m')]), OrderedDict([('attributes', OrderedDict([('type', 'Product2'), ('url', '/services/data/v57.0/sobjects/Product2/01t5j00000FRzZKAA1')])), ('Id', '01t5j00000FRzZKAA1'), ('Product_Id__c', '1aeb62f4-55e8-41d1-b920-dd01ded4da83'), ('Name', 'KW0005'), ('timestamp__c', '0000000000EB2E9D'), ('IsActive', True), ('Description', 'Green Scouring Pad 10 pack')]), OrderedDict([('attributes', OrderedDict([('type', 'Product2'), ('url', '/services/data/v57.0/sobjects/Product2/01t5j00000FRzZPAA1')])), ('Id', '01t5j00000FRzZPAA1'), ('Product_Id__c', '7998db97-149c-4964-90b6-83f8b2c6286c'), ('Name', 'KW0014'), ('timestamp__c', '0000000000EB2EC7'), ('IsActive', True), ('Description', '5 in 1 T-Shape Sponge Scouring Pad')]), OrderedDict([('attributes', OrderedDict([('type', 'Product2'), ('url', '/services/data/v57.0/sobjects/Product2/01t5j00000FRzZUAA1')])), ('Id', '01t5j00000FRzZUAA1'), ('Product_Id__c', 'fefe0999-244f-4ca6-95bc-07db3facede8'), ('Name', 'KW0013'), ('timestamp__c', '0000000000D56E41'), ('IsActive', True), ('Description', '2 in 1 Cleaning Brush')]), OrderedDict([('attributes', OrderedDict([('type', 'Product2'), ('url', '/services/data/v57.0/sobjects/Product2/01t5j00000FRzZZAA1')])), ('Id', '01t5j00000FRzZZAA1'), ('Product_Id__c', '1d0622c1-cddf-42e9-9775-c4744d5f4f12'), ('Name', 'KW1007'), ('timestamp__c', '00000000003E7404'), ('IsActive', True), ('Description', 'Basket KW1007')]), OrderedDict([('attributes', OrderedDict([('type', 'Product2'), ('url', '/services/data/v57.0/sobjects/Product2/01t5j00000FRzZeAAL')])), ('Id', '01t5j00000FRzZeAAL'), ('Product_Id__c', '865e1830-f10e-4b15-9b33-d4555b04dfb1'), ('Name', 'KW7710'), ('timestamp__c', '00000000003E73FE'), ('IsActive', True), ('Description', 'Leak Basin Set KW7710')]), OrderedDict([('attributes', OrderedDict([('type', 'Product2'), ('url', '/services/data/v57.0/sobjects/Product2/01t5j00000FRzZjAAL')])), ('Id', '01t5j00000FRzZjAAL'), ('Product_Id__c', '4ae5d5e1-b50a-43a8-9367-b1131c96fa9b'), ('Name', 'TV0001'), ('timestamp__c', '0000000000C8DF0D'), ('IsActive', True), ('Description', 'Universal Adapter')]), OrderedDict([('attributes', OrderedDict([('type', 'Product2'), ('url', '/services/data/v57.0/sobjects/Product2/01t5j00000FRzZoAAL')])), ('Id', '01t5j00000FRzZoAAL'), ('Product_Id__c', '839a0224-87b4-4f28-91a2-3c88a424bcef'), ('Name', 'GM0007'), ('timestamp__c', '0000000000BC7C32'), ('IsActive', True), ('Description', 'Plastic Coated Playing Card 2PK')]), OrderedDict([('attributes', OrderedDict([('type', 'Product2'), ('url', '/services/data/v57.0/sobjects/Product2/01t5j00000FRzZtAAL')])), ('Id', '01t5j00000FRzZtAAL'), ('Product_Id__c', '0ba36bcd-855f-482e-b8c9-efbc9e59b289'), ('Name', 'GM0001'), ('timestamp__c', '0000000000E3863E'), ('IsActive', True), ('Description', 'Pick-up reaching tool')]), OrderedDict([('attributes', OrderedDict([('type', 'Product2'), ('url', '/services/data/v57.0/sobjects/Product2/01t5j00000FRzZyAAL')])), ('Id', '01t5j00000FRzZyAAL'), ('Product_Id__c', '834600b5-5432-4c6c-adc7-600a155ca13c'), ('Name', 'OS0037'), ('timestamp__c', '0000000000F051A3'), ('IsActive', True), ('Description', '40mmx20m masking tape')]), OrderedDict([('attributes', OrderedDict([('type', 'Product2'), ('url', '/services/data/v57.0/sobjects/Product2/01t5j00000FRza3AAD')])), ('Id', '01t5j00000FRza3AAD'), ('Product_Id__c', '5f8bbafd-9fce-43a8-ac6a-e8c75c3c74fe'), ('Name', 'OS0017'), ('timestamp__c', '0000000000F05163'), ('IsActive', True), ('Description', 'Packing Tape 48mm x 50m')]), OrderedDict([('attributes', OrderedDict([('type', 'Product2'), ('url', '/services/data/v57.0/sobjects/Product2/01t5j00000FRza8AAD')])), ('Id', '01t5j00000FRza8AAD'), ('Product_Id__c', '8ddcb907-fe56-4917-8dc6-182ae7c80012'), ('Name', 'OS0033'), ('timestamp__c', '0000000000F05172'), ('IsActive', True), ('Description', '2PK Packing Tapes with dispenser')])]

processExistingOrder(inflowOrder, sfOrder,sfProducts)

[{'amountPaid': '112.02000', 'balance': '0.00000', 'billingAddress': {'address1': '135A Coode St', 'address2': '', 'city': 'South Perth', 'state': 'WA', 'country': 'Australia', 'postalCode': '6151', 'remarks': '', 'addressType': 'null'}, 'contactName': 'Van', 'dueDate': '2013-07-09T12:11:36.943+08:00', 'email': '', 'inventoryStatus': 'fulfilled', 'InvoiceDate': 'null', 'isCancelled': 'false', 'isCompleted': 'true', 'isInvoiced': 'true', 'isPrioritized': 'false', 'lines': [{'salesOrderLineId': '5facd02a-0ba0-4e05-bcb3-fcf22e9bfe9c', 'description': 'Toilet brush', 'discount': {'value': '0.00000', 'isPercent': 'true'}, 'isDiscarded': 'false', 'productId': '739f2977-a017-4106-9374-c37a2d0d9d98', 'quantity': {'standardQuantity': '3.0000', 'uomQuantity': '3.0000', 'uom': '', 'serialNumbers': []}, 'returnDate': 'null', 'serviceCompleted': 'null', 'subTotal': '5.67000', 'tax1Rate': '10.00000', 'tax2Rate': '0.00000', 'taxCodeId': '584d8a57-1a5d-4184-8f9d-e1ac94cf8f2b', 'timestamp': '00000000000

TypeError: list indices must be integers or slices, not str